In [ ]:
import altair as alt
import numpy as np
import pandas as pd

from mlrepricer import match, setup, helper, schemas
from mlrepricer.minmax import dump_csv
from mlrepricer.database import SQLite
alt.data_transformers.enable('default', max_rows=1000000)

In [ ]:
dump_csv()

In [ ]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}',
                       t.conn, parse_dates=[t.eventdate],
                       index_col='ID')

prime = df[(df.asin=='B01H0GZ2M2') & (df.isbuyboxwinner==1) & (df.isprime==1)]
notprime = df[(df.asin=='B01H0GZ2M2') & (df.isbuyboxwinner==1) & (df.isprime==0)]

In [ ]:
prime = prime.groupby(['time_changed', 'asin']).mean().reset_index('asin').sort_index().asfreq('24H', method='ffill').reset_index()
notprime = notprime.groupby(['time_changed', 'asin']).mean().reset_index('asin').sort_index().asfreq('30MIN').reset_index()

In [ ]:
prime

In [ ]:
alt.Chart(m).mark_rect().encode(
    x=alt.X('time_changed:T', bin=True),
    y=alt.Y('isprime:N'),
    color='isprime:N')

In [ ]:
square = alt.Chart(df).mark_square().encode(
    x=alt.X('time_changed:T', bin=True),
    y=alt.Y('price:Q', scale=alt.Scale(zero=False)),
    color='isprime:N',
    opacity=alt.value(1))

text = square.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=('price').format())
square.interactive()

In [ ]:
alt.Chart(m).mark_line().encode(
    x='time_changed:T',
    y='price').interactive()

In [ ]:
df.drop(['fulfillment-channel', 'count'], axis=1, inplace=True)